10

In [2]:
from sklearn.datasets import fetch_california_housing

In [3]:
housing = fetch_california_housing()

In [4]:
X = housing['data']
y = housing['target']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
import pandas as pd


In [8]:
X_train_df = pd.DataFrame(data=X_train, columns=housing['feature_names'])
X_train_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,3.2596,33.0,5.017657,1.006421,2300.0,3.691814,32.71,-117.03
1,3.8125,49.0,4.473545,1.041005,1314.0,1.738095,33.77,-118.16
2,4.1563,4.0,5.645833,0.985119,915.0,2.723214,34.66,-120.48
3,1.9425,36.0,4.002817,1.033803,1418.0,3.994366,32.69,-117.11
4,3.5542,43.0,6.268421,1.134211,874.0,2.300000,36.78,-119.80


In [9]:
X_train_df.describe()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000
mean,3.880754,28.608285,5.435235,1.096685,1426.453004,3.096961,35.643149,-119.582290
std,1.904294,12.602499,2.387375,0.433215,1137.056380,11.578744,2.136665,2.005654
min,0.499900,1.000000,0.888889,0.333333,3.000000,0.692308,32.550000,-124.350000
25%,2.566700,18.000000,4.452055,1.006508,789.000000,2.428799,33.930000,-121.810000
50%,3.545800,29.000000,5.235874,1.049286,1167.000000,2.817240,34.260000,-118.510000
75%,4.773175,37.000000,6.061037,1.100348,1726.000000,3.280000,37.720000,-118.010000
max,15.000100,52.000000,141.909091,25.636364,35682.000000,1243.333333,41.950000,-114.310000


In [10]:
from sklearn.preprocessing import StandardScaler


In [11]:
std_scalar = StandardScaler()
X_train_scaled_df = std_scalar.fit_transform(X_train_df)

- ### linear svr

In [12]:
from sklearn.svm import LinearSVR

In [13]:
lin_svr = LinearSVR(max_iter=100000, C = 10)
lin_svr.fit(X=X_train_scaled_df, y=y_train)

LinearSVR(C=10, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=100000,
          random_state=None, tol=0.0001, verbose=0)

In [14]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [15]:
y_train_pred = lin_svr.predict(X_train_scaled_df)
np.sqrt(mean_squared_error(y_train, y_train_pred))

0.9802098446872709

In [16]:
from sklearn.model_selection import cross_val_score

In [17]:
linear_svr_scores = cross_val_score(LinearSVR(C=5, max_iter=50000), X=X_train_scaled_df, y=y_train, cv=4, n_jobs=4, scoring="neg_mean_squared_error")
print(np.sqrt(-linear_svr_scores))

[0.74524508 0.72419182 3.57506498 0.84968896]


- ### poly svr

In [18]:
from sklearn.svm import SVR

In [19]:
poly_svr = SVR(kernel="poly", degree=3, C = 1, max_iter=100000)
poly_svr.fit(X_train_scaled_df, y_train)

/home/nimahsn/anaconda3/envs/tf2/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='poly', max_iter=100000, shrinking=True,
    tol=0.001, verbose=False)

In [20]:
y_train_pred = poly_svr.predict(X_train_scaled_df)
mean_squared_error(y_train, y_train_pred)

2.7734981684779294

SUCKS!

- ### rbf svr

In [21]:
# y_train
rbf_svr = SVR(C=5)
rbf_svr.fit(X_train_scaled_df, y_train)

SVR(C=5, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [22]:
y_train_pred = rbf_svr.predict(X_train_scaled_df)
np.sqrt(mean_squared_error(y_train, y_train_pred))


0.5528948745649199

In [23]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, n_iter=10, verbose=2, cv=3, random_state=42, n_jobs=6)
rnd_search_cv.fit(X_train_scaled_df, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  30 out of  30 | elapsed:  1.5min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                 epsilon=0.1, gamma='auto_deprecated',
                                 kernel='rbf', max_iter=-1, shrinking=True,
                                 tol=0.001, verbose=False),
                   iid='warn', n_iter=10, n_jobs=6,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fadecae16d0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fad97e9a290>},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

In [25]:
preds = rnd_search_cv.best_estimator_.predict(X_train_scaled_df)
np.mean(mean_squared_error(y_train, preds))

0.3280453999995985

In [26]:
rnd_search_cv.best_estimator_.fit(X_train_scaled_df, y_train)

SVR(C=4.745401188473625, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=0.07969454818643928, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [29]:
preds = rnd_search_cv.best_estimator_.predict(X_train_scaled_df)
np.sqrt(mean_squared_error(y_train, preds))

0.5727524770785356